<font size="6">  
Import packages
</font>

In [2]:
import torch
import torchvision
import pandas as pd
import pickle 
from itertools import compress
from ultralytics import YOLO
from PIL import Image

<font size="6">  
Check for cuda (optional?)
</font>

In [3]:
torch.cuda.is_available()
torchvision.__version__ 
torch.__version__ 

'2.2.1+cu121'

<font size="6">  
Import weights from best run
</font>

In [6]:
# 1) Import weights into YOLOv8
# NOTE: change path to wherever your weights are stored
trained = YOLO("C:/Users/gener/Desktop/Refrigerator/train6/weights/best.pt")

# 2) Run prediction on chosen test image
test_image = "veggies2.jpg"
results = trained(test_image, conf=0.01, save_txt=True)

# 3) Print and save test image
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image
    im_rgb.save("bound_box_result.jpg")
    # Show results to screen (in supported environments)
    # r.show()



image 1/1 C:\Users\gener\Desktop\Refrigerator\veggies2.jpg: 384x640 1 Bell pepper, 1 Cabbage, 1 Chili pepper, 1 Corn, 1 Garlic, 1 Radish, 1 Tomato, 42.4ms
Speed: 8.1ms preprocess, 42.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\Users\gener\PycharmProjects\pangolin\runs\detect\predict7
1 label saved to C:\Users\gener\PycharmProjects\pangolin\runs\detect\predict7\labels


<font size="6">  
Convert result into usable string for Ingredient Lookup
</font>

In [12]:
# Extract text file info
class_list = []

# 1) Parse text file for bounding box labels
# NOTE: Change directory to where .txt is stored
with open('C:/Users/gener/PycharmProjects/pangolin/runs/detect/predict2/labels/veggies2.txt') as f:
    lines = f.readlines() 
    for line in lines:
        line = line.strip() 
        splitted_line = line.split()
        class_list.append(splitted_line[0]) 
        
results_dict = results[0].names

# 2) Convert into ingredient counts dictionary
ingredients_dict = {}
for i in class_list:
    index = int(i)
    ingred_name = results_dict[index]
    if ingred_name not in ingredients_dict:
        ingredients_dict[ingred_name] = 0
    ingredients_dict[ingred_name] += 1
    
print(ingredients_dict)

# 3) Convert ingredient counts dictionary into comma-separated string
print('\n')
food = ''
first = True
for ing in ingredients_dict.keys():
    if not first:
        food += ','
    food += ing.lower()
    first = False
    j += 1
print(food)

{'Tomato': 1, 'Bell pepper': 1, 'Radish': 1, 'Corn': 1, 'Cabbage': 1, 'Garlic': 1, 'Chili pepper': 1}


tomato,bell pepper,radish,corn,cabbage,garlic,chili pepper


<font size="6">
    Functions for Ingredient Search
</font>

In [ ]:
d = None
df = None
freq = {}

def process_1K():
    df = pd.read_csv('1K_dataset.csv')
    df["IGS"] = df["NER"].apply(lambda x: set(x.lower().replace('[', '').replace(']', '').replace('\"', '').replace(', ', ',').split(",")))
    d = df["IGS"].values
    np.save('1K_IGS', d)
    return d

def process_full():
    df = pd.read_csv('full_dataset.csv')
    df["IGS"] = df["NER"].apply(lambda x: set(x.lower().replace('[', '').replace(']', '').replace('\"', '').replace(', ', ',').split(",")))
    d = df["IGS"].values
    np.save('full_IGS', d)
    return d

def process_optimized_1K():
    df = pd.read_csv('1K_dataset.csv')
    freq = {}
    for i in range(df.shape[0]):
        for ing in df["NER"][i].lower().replace('[', '').replace(']', '').replace('\"', '').replace(', ', ',').split(","):
            if ing in freq: freq[ing].append(i)
            else: freq[ing] = [i]
        print(i)
    with open('prcomputed_indices_1K.pkl', 'wb') as f:
        pickle.dump(freq, f)
    return df, freq

def process_optimized_full():
    df = pd.read_csv('full_dataset.csv')
    freq = {}
    for i in range(df.shape[0]):
        for ing in df["NER"][i].lower().replace('[', '').replace(']', '').replace('\"', '').replace(', ', ',').split(","):
            if ing in freq: freq[ing].append(i)
            else: freq[ing] = [i]
#         print(i)
    with open('prcomputed_indices_full.pkl', 'wb') as f:
        pickle.dump(freq, f)
    return df, freq

def initialize_1K():
    df = pd.read_csv('1K_dataset.csv')
    d = np.load('1k_IGS.npy', allow_pickle=True)
    return df, d

def initialize_full():
    df = pd.read_csv('full_dataset.csv')
    d = np.load('full_IGS.npy', allow_pickle=True)
    return df, d

def initialize_optimized_1K():
    df = pd.read_csv('full_dataset.csv')
    with open('prcomputed_indices_1K.pkl', 'rb') as f:
        freq = pickle.load(f)
    return df, freq

def initialize_optimized_full():
    df = pd.read_csv('full_dataset.csv')
    with open('prcomputed_indices_full.pkl', 'rb') as f:
        freq = pickle.load(f)
    return df, freq


def get_indices(ing_list):
    ingredients = set(ing_list)
    return np.where([ingredients.issubset(x) for x in d])

def get_indices_optimized(ing_list):
    if set(ing_list).issubset(freq.keys()):
        base = freq[ing_list[0]]
        for ing in ing_list:
            base = np.intersect1d(base, freq[ing])
        return base
    else: return []

def get_recipes(ing_list):
    indices = get_indices(ing_list)
    return df.loc[indices[0], ["title", "link"]]

def get_recipes_optimized(ing_list):
    indices = get_indices_optimized(ing_list)
    return df.loc[indices, ["title", "link"]]


<font size="6">
    Run Ingredient Search
</font>

In [ ]:
# NOTE: Can just run once (takes ~3 mins to run)
process_optimized_full()
print("process optimized done")
df, freq = initialize_optimized_full()
print("initialization done")

In [ ]:
print(get_recipes_optimized(food.split(",")))